Implement Perceptron using PyTorch

In [28]:
# 3-26-2018
# Brent DeVetter
# Written for the Google CoLab environment 
#   - Uses PyTorch and requires a GPU (as written)
#   - Reference: https://medium.com/@tomgrek/building-your-first-neural-net-from-scratch-with-pytorch-56b0e9c84d54

# Import libaries
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

import numpy as np
import torch
import torch.nn.functional as F
from torch import autograd, nn, optim
from torch.autograd import Variable

# Loss function (mean-squared error, MSE)
def criterion(out, label):
  return (label - out)**2

# Set up a neural network with a bias (b) and weights (A)
# No activation function is used here
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()                                                 # pytorch requires this statement
    self.fc1 = nn.Linear(in_features=1, out_features=1, bias=True)              # Linear transformation: i.e., y = Ax + b
    
  def forward(self, x):
    x = self.fc1(x)
    return x   
    
if __name__ == "__main__":
  # Create Net instance and print properties
  net = Net().cuda()
  print(net)
  print(list(net.parameters()))
  
  # Create a random number for the net
  input = Variable(torch.randn(1,1,1), requires_grad=True).cuda()
  print("Input: {0}".format(input))
  
  # Test that the network is working
  out = net(input)
  print(out)
  
  # Now that we have a working net, train the data. 
  # Dataset we are trying to train to Ax + b (ie, A = 3, b = 0)
  data = [(1,3), (2,6), (3,9), (4,12), (5,15), (6,18)]

  optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)
  
  for epoch in range(101):
    for i, data2 in enumerate(data):
      X, Y = iter(data2)
      X, Y = Variable(torch.FloatTensor([X]), requires_grad=True).cuda(), Variable(torch.FloatTensor([Y]), requires_grad=False).cuda()
        
      optimizer.zero_grad() # This is required don't forget to zero the gradient
      y_pred = net(X)
      loss = criterion(y_pred, Y)
      loss.backward()
      optimizer.step()
    if (epoch % 10 == 0):
      print("Epoch {0} - loss: {1}".format(epoch, loss.data[0]))   
        
        
print(list(net.parameters()))        
        

Net(
  (fc1): Linear(in_features=1, out_features=1)
)
[Parameter containing:
 0.8926
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]
, Parameter containing:
 0.1601
[torch.cuda.FloatTensor of size 1 (GPU 0)]
]
Input: Variable containing:
(0 ,.,.) = 
  1.0524
[torch.cuda.FloatTensor of size 1x1x1 (GPU 0)]

Variable containing:
(0 ,.,.) = 
  1.0995
[torch.cuda.FloatTensor of size 1x1x1 (GPU 0)]

Epoch 0 - loss: 0.026212086901068687
Epoch 10 - loss: 0.04181097075343132
Epoch 20 - loss: 0.01461385004222393
Epoch 30 - loss: 0.0051082707941532135
Epoch 40 - loss: 0.0017856801860034466
Epoch 50 - loss: 0.0006241229129955173
Epoch 60 - loss: 0.00021822424605488777
Epoch 70 - loss: 7.624506542924792e-05
Epoch 80 - loss: 2.6678259018808603e-05
Epoch 90 - loss: 9.313225746154785e-06
Epoch 100 - loss: 3.2556854421272874e-06
[Parameter containing:
 2.9993
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]
, Parameter containing:
1.00000e-03 *
  4.0758
[torch.cuda.FloatTensor of size 1 (GPU 0)]
]


In [71]:
# Now, use a ReLU activation function AND a multi-layer perceptron 
# Note that I found it necessary to change the SGD lr and momentum (especially lr, otherwise massive error was present)

# Loss function (mean-squared error, MSE)
def criterion(out, label):
  return (label - out)**2

# Set up a neural network with a bias (b) and weights (A)
# No activation function is used here
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()                                                 # pytorch requires this statement
    self.fc1 = nn.Linear(in_features=1, out_features=3, bias=True)             # Linear transformation: i.e., y = Ax + b
    self.fc2 = nn.Linear(in_features=3, out_features=1, bias=True)
  def forward(self, x):
    x = self.fc2(F.relu(self.fc1(x)))
    return x   
    
if __name__ == "__main__":
  # Create Net instance and print properties
  net = Net().cuda()
  print(net)
  print(list(net.parameters()))
  
  # Create a random number for the net
  input = Variable(torch.randn(1,1,1), requires_grad=True).cuda()
  print("Input: {0}".format(input))
  
  # Test that the network is working
  out = net(input)
  print(out)
  
  # Now that we have a working net, train the data. 
  # Dataset we are trying to train to Ax + b (ie, A = 3, b = 0)
  data = [(1,3), (2,6), (3,9), (4,12), (5,15), (6,18)]

  optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.6)
  
  for epoch in range(101):
    for i, data2 in enumerate(data):
      X, Y = iter(data2)
      X, Y = Variable(torch.FloatTensor([X]), requires_grad=True).cuda(), Variable(torch.FloatTensor([Y]), requires_grad=False).cuda()
        
      optimizer.zero_grad() # This is required don't forget to zero the gradient
      y_pred = net(X)
      loss = criterion(y_pred, Y)
      loss.backward()
      optimizer.step()
    if (epoch % 10 == 0):
      print("Epoch {0} - loss: {1}".format(epoch, loss.data[0]))   
        
print("Optimized parameters: ")
print(list(net.parameters()))        
        
print("Prediction: ")  
print(net(Variable(torch.Tensor([[[1]]]).cuda())))  

Net(
  (fc1): Linear(in_features=1, out_features=3)
  (fc2): Linear(in_features=3, out_features=1)
)
[Parameter containing:
 0.5437
 0.4377
-0.1207
[torch.cuda.FloatTensor of size 3x1 (GPU 0)]
, Parameter containing:
 0.9994
 0.0670
-0.0591
[torch.cuda.FloatTensor of size 3 (GPU 0)]
, Parameter containing:
-0.4058 -0.1300  0.3575
[torch.cuda.FloatTensor of size 1x3 (GPU 0)]
, Parameter containing:
1.00000e-02 *
  4.7812
[torch.cuda.FloatTensor of size 1 (GPU 0)]
]
Input: Variable containing:
(0 ,.,.) = 
  0.7672
[torch.cuda.FloatTensor of size 1x1x1 (GPU 0)]

Variable containing:
(0 ,.,.) = 
 -0.5794
[torch.cuda.FloatTensor of size 1x1x1 (GPU 0)]

Epoch 0 - loss: 307.56732177734375
Epoch 10 - loss: 0.7161722779273987
Epoch 20 - loss: 0.4294607639312744
Epoch 30 - loss: 0.2592196762561798
Epoch 40 - loss: 0.15642793476581573
Epoch 50 - loss: 0.09409399330615997
Epoch 60 - loss: 0.056354258209466934
Epoch 70 - loss: 0.033598896116018295
Epoch 80 - loss: 0.01994742453098297
Epoch 90 - los